In [12]:
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
path = './data/data_with_time_series.xlsx'
df = pd.read_excel(path)
df.shape

(3629, 30)

In [3]:
df_ori = df.filter(['手术ID','病区','实施手术_分词','术前诊断_分词']).copy()
df_ori.shape

(3629, 4)

In [71]:
# 返回以列表形式表示的分词结果
def word_list(s):
    return s.split('|')


def word_dictionary(df,col):   
    ans = set()
    def handle(s):
        for w in word_list(s):
            ans.add(w)
    df[col].apply(handle)
    print('获得单词:'+str(len(ans)))
    return ans

def get_departments(df):
    return list(set(df['病区']))

In [84]:
# 计算每一个单词的卡方值
def CHI(df,word,department,word_col_name):
    N = df.shape[0]
    A = df[(df['病区'] == department) & (df[word_col_name].str.contains(word))].shape[0]
    B = df[~(df['病区'] == department) & (df[word_col_name].str.contains(word))].shape[0]
    C = df[(df['病区'] == department) & ~(df[word_col_name].str.contains(word))].shape[0]
    D = df[~(df['病区'] == department) & ~(df[word_col_name].str.contains(word))].shape[0]
#     print(N,A,B,C,D)
    numerator = N*((A*D - B*C)**2)
    denominator = (A+B)*(A+C)*(D+B)*(D+C)
    return round(numerator / denominator,4)

# 计算feature_chi_matrix矩阵
def get_feature_chi_matrix(df,departments,words,word_col_name):
    feature_chi_matrix = pd.DataFrame(index=words,columns=departments,dtype='float')
    count = 1
    for w in words:
        if count % 100 == 0:
            print('进度:' + str(count))
        row = []
        for c in departments:
            row.append(CHI(df,w,c,word_col_name))
        feature_chi_matrix.loc[w] = row
        count += 1
    feature_chi_matrix['CHI_MAX'] = feature_chi_matrix.max(axis=1)
    feature_chi_matrix.sort_values('CHI_MAX',ascending=False,inplace=True)
    return feature_chi_matrix


In [10]:
dictionary = word_dictionary(df_ori,'术前诊断_分词')

In [50]:
cols = get_departments(df_ori)
cols

['肝胆外科病区',
 '妇产科病区',
 '普通外科病区',
 '心血管病区',
 '眼科病区',
 '整形外科病区',
 '泌尿外科中心病区',
 '骨科病区',
 '耳鼻咽喉科病区',
 '口腔科病区',
 '神经内科病区',
 '神经外科病区',
 '普通胸外科病区']

In [52]:
feature_chi_matrix = get_feature_chi_matrix(df_ori,cols,dictionary,'术前诊断_分词')

3629 0 1 234 3394
3629 1 0 598 3030
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 1 234 3394
3629 1 0 598 3030
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 7 234 3388
3629 0 7 599 3023
3629 0 7 317 3305
3629 0 7 363 3259
3629 0 7 21 3601
3629 0 7 21 3601
3629 7 0 411 3211
3629 0 7 809 2813
3629 0 7 215 3407
3629 0 7 90 3532
3629 0 7 58 3564
3629 0 7 290 3332
3629 0 7 194 3428
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 1 0 289 3339
3629 0 1 194 3434
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1

3629 0 7 317 3305
3629 0 7 363 3259
3629 0 7 21 3601
3629 0 7 21 3601
3629 7 0 411 3211
3629 0 7 809 2813
3629 0 7 215 3407
3629 0 7 90 3532
3629 0 7 58 3564
3629 0 7 290 3332
3629 0 7 194 3428
3629 0 58 234 3337
3629 0 58 599 2972
3629 1 57 316 3255
3629 4 54 359 3212
3629 0 58 21 3550
3629 0 58 21 3550
3629 0 58 418 3153
3629 0 58 809 2762
3629 0 58 215 3356
3629 0 58 90 3481
3629 41 17 17 3554
3629 12 46 278 3293
3629 0 58 194 3377
3629 1 15 233 3380
3629 0 16 599 3014
3629 0 16 317 3296
3629 0 16 363 3250
3629 0 16 21 3592
3629 0 16 21 3592
3629 4 12 414 3199
3629 8 8 801 2812
3629 0 16 215 3398
3629 0 16 90 3523
3629 0 16 58 3555
3629 0 16 290 3323
3629 3 13 191 3422
3629 0 2 234 3393
3629 0 2 599 3028
3629 0 2 317 3310
3629 0 2 363 3264
3629 0 2 21 3606
3629 0 2 21 3606
3629 0 2 418 3209
3629 2 0 807 2820
3629 0 2 215 3412
3629 0 2 90 3537
3629 0 2 58 3569
3629 0 2 290 3337
3629 0 2 194 3433
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607


3629 0 2 194 3433
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 1 0 20 3608
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 1 0 417 3211
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 2 234 3393
3629 2 0 597 3030
3629 0 2 317 3310
3629 0 2 363 3264
3629 0 2 21 3606
3629 0 2 21 3606
3629 0 2 418 3209
3629 0 2 809 2818
3629 0 2 215 3412
3629 0 2 90 3537
3629 0 2 58 3569
3629 0 2 290 3337
3629 0 2 194 3433
3629 0 5 234 3390
3629 2 3 597 3027
3629 0 5 317 3307
3629 2 3 361 3263
3629 0 5 21 3603
3629 0 5 21 3603
3629 0 5 418 3206
3629 1 4 808 2816
3629 0 5 215 3409
3629 0 5 90 3534
3629 0 5 58 3566
3629 0 5 290 3334
3629 0 5 194 3430
3629 3 43 231 3352
3629 13 33 586 2997
3629 6 40 311 3272
3629

3629 0 115 58 3456
3629 0 115 290 3224
3629 5 110 189 3325
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 1 0 808 2820
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 1 0 362 3266
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 17 69 217 3326
3629 17 69 582 2961
3629 1 85 316 3227
3629 0 86 363 3180
3629 0 86 21 3522
3629 0 86 21 3522
3629 2 84 416 3127
3629 5 81 804 2739
3629 42 44 173 3370
3629 0 86 90 3453
3629 0 86 58 3485
3629 1 85 289 3254
3629 1 85 193 3350
3629 0 2 234 3393
3629 0 2 599 3028
3629 0 2 317 3310
3629 0 2 363 3264
3629 0 2 21 3606
3629 0 2 21 3606
3629 0 2 418 3209
3629 1 1 808 2819
3629 0 2 215 3412
3629 1 1 89 3538
3629 0 2 58 3569
3629 0 2 290 3337
3629 0 2 194 3433
3629 

3629 0 6 21 3602
3629 0 6 418 3205
3629 1 5 808 2815
3629 5 1 210 3413
3629 0 6 90 3533
3629 0 6 58 3565
3629 0 6 290 3333
3629 0 6 194 3429
3629 0 10 234 3385
3629 10 0 589 3030
3629 0 10 317 3302
3629 0 10 363 3256
3629 0 10 21 3598
3629 0 10 21 3598
3629 0 10 418 3201
3629 0 10 809 2810
3629 0 10 215 3404
3629 0 10 90 3529
3629 0 10 58 3561
3629 0 10 290 3329
3629 0 10 194 3425
3629 0 11 234 3384
3629 6 5 593 3025
3629 0 11 317 3301
3629 3 8 360 3258
3629 0 11 21 3597
3629 0 11 21 3597
3629 0 11 418 3200
3629 0 11 809 2809
3629 1 10 214 3404
3629 0 11 90 3528
3629 0 11 58 3560
3629 1 10 289 3329
3629 0 11 194 3424
3629 0 141 234 3254
3629 28 113 571 2917
3629 7 134 310 3178
3629 64 77 299 3189
3629 1 140 20 3468
3629 0 141 21 3467
3629 9 132 409 3079
3629 22 119 787 2701
3629 0 141 215 3273
3629 0 141 90 3398
3629 6 135 52 3436
3629 4 137 286 3202
3629 0 141 194 3294
3629 0 5 234 3390
3629 0 5 599 3025
3629 0 5 317 3307
3629 0 5 363 3261
3629 0 5 21 3603
3629 0 5 21 3603
3629 0 5 41

...

3629 5 11 594 3019
3629 6 10 311 3302
3629 0 16 363 3250
3629 0 16 21 3592
3629 0 16 21 3592
3629 0 16 418 3195
3629 0 16 809 2804
3629 4 12 211 3402
3629 0 16 90 3523
3629 0 16 58 3555
3629 0 16 290 3323
3629 0 16 194 3419
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 1 0 289 3339
3629 0 1 194 3434
3629 0 1 234 3394
3629 0 1 599 3029
3629 1 0 316 3312
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 2 234 3393
3629 0 2 599 3028
3629 0 2 317 3310
3629 0 2 363 3264
3629 0 2 21 3606
3629 0 2 21 3606
3629 2 0 416 3211
3629 0 2 809 2818
3629 0 2 215 3412
3629 0 2 90 3537
3629 0 2 58 3569
3629 0 2 290 3337
3629 0 2 194 3433
3629 0 4 234 3391
3629 0 4 599 3026
3629 0 4 317 3308
3629 0 4 363 3262
3629 0 4 21 36

3629 0 11 317 3301
3629 4 7 359 3259
3629 0 11 21 3597
3629 0 11 21 3597
3629 0 11 418 3200
3629 1 10 808 2810
3629 0 11 215 3403
3629 0 11 90 3528
3629 2 9 56 3562
3629 1 10 289 3329
3629 0 11 194 3424
3629 0 4 234 3391
3629 0 4 599 3026
3629 0 4 317 3308
3629 0 4 363 3262
3629 1 3 20 3605
3629 1 3 20 3605
3629 1 3 417 3208
3629 1 3 808 2817
3629 0 4 215 3410
3629 0 4 90 3535
3629 0 4 58 3567
3629 0 4 290 3335
3629 0 4 194 3431
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 1 0 214 3414
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 3 0 231 3395
3629 0 3 599 3027
3629 0 3 317 3309
3629 0 3 363 3263
3629 0 3 21 3605
3629 0 3 21 3605
3629 0 3 418 3208
3629 0 3 809 2817
3629 0 3 215 3411
3629 0 3 90 3536
3629 0 3 58 3568
3629 0 3 290 3336
3629 0 3 194 3432
3629 1 0 233 3395
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3

3629 9 11 590 3019
3629 1 19 316 3293
3629 0 20 363 3246
3629 0 20 21 3588
3629 0 20 21 3588
3629 1 19 417 3192
3629 1 19 808 2801
3629 0 20 215 3394
3629 2 18 88 3521
3629 0 20 58 3551
3629 6 14 284 3325
3629 0 20 194 3415
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 1 0 808 2820
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 6 234 3389
3629 0 6 599 3024
3629 0 6 317 3306
3629 0 6 363 3260
3629 1 5 20 3603
3629 0 6 21 3602
3629 1 5 417 3206
3629 0 6 809 2814
3629 4 2 211 3412
3629 0 6 90 3533
3629 0 6 58 3565
3629 0 6 290 3333
3629 0 6 194 3429
3629 0 44 234 3351
3629 2 42 597 2988
3629 0 44 317 3268
3629 0 44 363 3222
3629 0 44 21 3564
3629 0 44 21 3564
3629 42 2 376 3209
3629 0 44 809 2776
3629 0 44 215 3370
3629 0 44 90 3495
3629 0 44 58 3527
3629 0 44 290 3295
3629 0 44 194 3391
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3

3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 1 0 214 3414
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 1 0 214 3414
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 2 234 3393
3629 0 2 599 3028
3629 0 2 317 3310
3629 0 2 363 3264
3629 0 2 21 3606
3629 0 2 21 3606
3629 0 2 418 3209
3629 0 2 809 2818
3629 0 2 215 3412
3629 0 2 90 3537
3629 0 2 58 3569
3629 2 0 288 3339
3629 0 2 194 3433
3629 0 2 234 3393
3629 0 2 599 3028
3629 0 2 317 3310
3629 0 2 363 3264
3629 0 2 21 3606
3629 0 2 21 3606
3629 0 2 418 3209
3629 2 0 807 2820
3629 0 2 215 3412
3629 0 2 90 3537
3629 0 2 58 3569
3629 0 2 290 3337
3629 0 2 194 3433
3629 0 3 234 3392
3629 0 3 599 3027
3629 0 3 317 3309
3629 0 3 363 3263
3629 0 3 21 3605
3629 0 3 21 3605
3629 0 3 4

3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 1 0 417 3211
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 1 234 3394
3629 0 1 599 3029
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 1 0 289 3339
3629 0 1 194 3434
3629 2 0 232 3395
3629 0 2 599 3028
3629 0 2 317 3310
3629 0 2 363 3264
3629 0 2 21 3606
3629 0 2 21 3606
3629 0 2 418 3209
3629 0 2 809 2818
3629 0 2 215 3412
3629 0 2 90 3537
3629 0 2 58 3569
3629 0 2 290 3337
3629 0 2 194 3433
3629 0 1 234 3394
3629 1 0 598 3030
3629 0 1 317 3311
3629 0 1 363 3265
3629 0 1 21 3607
3629 0 1 21 3607
3629 0 1 418 3210
3629 0 1 809 2819
3629 0 1 215 3413
3629 0 1 90 3538
3629 0 1 58 3570
3629 0 1 290 3338
3629 0 1 194 3434
3629 0 24 234 3371
3629 0 24 599 3006
3629 0 24 317 3288
3629 0 24 363 3242
3629 0 24 21 3584
3629

In [53]:
feature_chi_matrix.head()

,肝胆外科病区,妇产科病区,普通外科病区,心血管病区,眼科病区,整形外科病区,泌尿外科中心病区,骨科病区,耳鼻咽喉科病区,口腔科病区,神经内科病区,神经外科病区,普通胸外科病区
松解术,0.0689,5.0598,0.0957,0.1112,0.0058,0.0058,0.1302,0.2870,0.0630,0.0254,0.0162,0.0869,0.0565
单纯性,0.0689,5.0598,0.0957,0.1112,0.0058,0.0058,0.1302,0.2870,0.0630,0.0254,0.0162,0.0869,0.0565
双肾,0.4834,1.3865,0.6713,0.7795,0.0408,0.0408,53.8767,2.0120,0.4417,0.1784,0.1139,0.6091,0.3961
颅咽,0.0689,0.1977,0.0957,0.1112,0.0058,0.0058,0.1302,0.2870,0.0630,0.0254,0.0162,11.5170,0.0565
内翻,0.0689,0.1977,0.0957,0.1112,0.0058,0.0058,0.1302,3.4867,0.0630,0.0254,0.0162,0.0869,0.0565


In [54]:
CHI(df_ori,'松解术','肝胆外科病区','术前诊断_分词')

3629 0 1 234 3394


0.0689

In [56]:
feature_chi_matrix.max(axis=1)

松解术      5.0598
单纯性      5.0598
双肾      53.8767
颅咽      11.5170
内翻       3.4867
         ...   
稍        7.6839
粘膜       6.8728
旋转       7.6839
高脂血症    59.6510
拇指       3.4867
Length: 1690, dtype: float64

In [57]:
feature_chi_matrix['CHI_MAX'] = feature_chi_matrix.max(axis=1)
feature_chi_matrix.head()

,肝胆外科病区,妇产科病区,普通外科病区,心血管病区,眼科病区,整形外科病区,泌尿外科中心病区,骨科病区,耳鼻咽喉科病区,口腔科病区,神经内科病区,神经外科病区,普通胸外科病区,CHI_MAX
松解术,0.0689,5.0598,0.0957,0.1112,0.0058,0.0058,0.1302,0.2870,0.0630,0.0254,0.0162,0.0869,0.0565,5.0598
单纯性,0.0689,5.0598,0.0957,0.1112,0.0058,0.0058,0.1302,0.2870,0.0630,0.0254,0.0162,0.0869,0.0565,5.0598
双肾,0.4834,1.3865,0.6713,0.7795,0.0408,0.0408,53.8767,2.0120,0.4417,0.1784,0.1139,0.6091,0.3961,53.8767
颅咽,0.0689,0.1977,0.0957,0.1112,0.0058,0.0058,0.1302,0.2870,0.0630,0.0254,0.0162,11.5170,0.0565,11.5170
内翻,0.0689,0.1977,0.0957,0.1112,0.0058,0.0058,0.1302,3.4867,0.0630,0.0254,0.0162,0.0869,0.0565,3.4867


In [63]:
l = list(feature_chi_matrix['CHI_MAX'].sort_values(ascending=False))

In [75]:
dictionary_operation = word_dictionary(df_ori,'实施手术_分词')

获得单词:1343


In [76]:
feature_chi_matrix_operation = get_feature_chi_matrix(df_ori,cols,dictionary_operation,'实施手术_分词')

In [78]:
feature_chi_matrix_operation.head()

,肝胆外科病区,妇产科病区,普通外科病区,心血管病区,眼科病区,整形外科病区,泌尿外科中心病区,骨科病区,耳鼻咽喉科病区,口腔科病区,神经内科病区,神经外科病区,普通胸外科病区,CHI_MAX
松解术,1.1148,314.4115,10.4091,18.3603,0.9615,0.9615,14.9989,18.8639,4.8035,4.2010,2.6830,14.3473,9.3296,314.4115
矫正,5.9269,16.9994,8.2304,9.5574,26.1555,0.5595,11.1940,4.1991,868.4951,2.1868,1.3967,7.4685,4.8565,868.4951
单纯性,0.1379,0.3956,4.2742,0.2224,0.0116,0.0116,2.9076,0.5741,0.1260,0.0509,0.0325,0.1738,0.1130,4.2742
切断术,0.7605,2.1812,1.0560,1.2263,0.0642,0.0642,1.4363,3.1653,0.6948,0.2806,0.1792,0.9583,195.3602,195.3602
管髓,0.0689,0.1977,0.0957,0.1112,0.0058,0.0058,0.1302,3.4867,0.0630,0.0254,0.0162,0.0869,0.0565,3.4867


In [79]:
feature_chi_matrix_operation.shape

(1343, 14)

In [80]:
feature_chi_matrix_operation.sort_values('CHI_MAX',ascending=False,inplace=True)
feature_chi_matrix_operation.head()

,肝胆外科病区,妇产科病区,普通外科病区,心血管病区,眼科病区,整形外科病区,泌尿外科中心病区,骨科病区,耳鼻咽喉科病区,口腔科病区,神经内科病区,神经外科病区,普通胸外科病区,CHI_MAX
胸腔,12.2931,35.2590,17.0708,19.8233,1.0381,1.0381,23.2178,51.1664,11.2321,4.5357,2.8968,15.4906,3157.9870,3157.9870
胸腔镜,12.2173,35.0414,16.9655,19.7010,1.0317,1.0317,23.0746,50.8507,11.1628,4.5077,2.8790,15.3950,3138.5033,3138.5033
断术,9.7055,33.9556,16.4398,19.0905,0.9997,0.9997,22.3596,46.6163,10.8169,2.4842,2.7898,14.9180,2924.8398,2924.8398
肺,11.5366,33.0892,16.0203,8.7952,0.9742,0.9742,21.7891,48.0178,10.5409,4.2566,2.7186,14.5373,2771.1958,2771.1958
胸膜,10.7845,30.9320,14.9759,17.3906,0.9107,0.9107,20.3685,44.8873,9.8537,3.9791,2.5413,13.5896,2770.4404,2770.4404


In [81]:
feature_chi_matrix.sort_values('CHI_MAX',ascending=False,inplace=True)
feature_chi_matrix.head()

,肝胆外科病区,妇产科病区,普通外科病区,心血管病区,眼科病区,整形外科病区,泌尿外科中心病区,骨科病区,耳鼻咽喉科病区,口腔科病区,神经内科病区,神经外科病区,普通胸外科病区,CHI_MAX
眼,1.9449,3.4255,2.7008,3.1363,2462.0024,0.1642,3.6733,5.7091,1.7770,0.7176,0.4583,6.9296,1.5937,2462.0024
胆囊,2175.7908,28.2673,13.8098,18.6034,0.9742,0.9742,15.2726,48.0178,8.4342,4.2566,1.0081,14.5373,9.4532,2175.7908
心,13.6209,38.3854,25.3807,2130.6636,1.6842,1.6842,28.5396,80.2585,18.2233,7.3589,0.0206,20.6576,16.3429,2130.6636
宫,28.6541,2009.4694,30.6204,46.2064,2.4197,2.4197,49.1974,119.2644,26.1810,10.5724,4.6769,36.1071,23.4794,2009.4694
肺,15.1305,31.6251,7.8920,1.5952,1.2777,1.2777,14.2603,55.0541,13.8246,3.6195,3.5655,6.3438,1982.5742,1982.5742


In [82]:
feature_chi_matrix_operation.to_excel('./feature_matrix/chi_feature_operation.xlsx')

In [83]:
feature_chi_matrix.to_excel('./feature_matrix/chi_feature_diagnose.xlsx')